In [7]:
pip install numpy
pip install h5py
pip install rdkit
pip install py3Dmol

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 8.7 MB/s  0:00:00m eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h5py
import time
import numpy as np
import json
import pymsym
import ira_mod
sofi = ira_mod.SOFI()

def save_data_hdf5(filename, all_atom_types, all_coords, all_smiles, all_selfies):
    """
    Save data using HDF5 - excellent for very large datasets
    """
    print(f"Starting to save {len(all_atom_types)} molecules to {filename}...")
    start_time = time.time()
    
    with h5py.File(filename, 'w') as f:
        # Save coordinates as variable-length arrays
        print("Saving coordinates...")
        coord_dt = h5py.special_dtype(vlen=np.dtype('float32'))
        coord_dset = f.create_dataset('coords', (len(all_coords),), dtype=coord_dt, 
                                     compression='gzip', compression_opts=9)
        
        for i, coords in enumerate(all_coords):
            if i % 10000 == 0:  # Progress indicator
                print(f"  Processed {i}/{len(all_coords)} coordinates...")
            coord_dset[i] = np.array(coords, dtype=np.float32).flatten()
        
        # Save atom types as variable-length strings
        print("Saving atom types...")
        atom_dt = h5py.special_dtype(vlen=str)
        atom_dset = f.create_dataset('atom_types', (len(all_atom_types),), dtype=atom_dt,
                                   compression='gzip', compression_opts=9)
        
        for i, atoms in enumerate(all_atom_types):
            if i % 10000 == 0:
                print(f"  Processed {i}/{len(all_atom_types)} atom types...")
            atom_dset[i] = '|'.join(atoms)  # Join with delimiter
        
        # Save SMILES and SELFIES
        print("Saving SMILES...")
        smiles_dset = f.create_dataset('smiles', (len(all_smiles),), dtype=atom_dt,
                                     compression='gzip', compression_opts=9)
        
        print("Saving SELFIES...")
        selfies_dset = f.create_dataset('selfies', (len(all_selfies),), dtype=atom_dt,
                                      compression='gzip', compression_opts=9)
        
        for i in range(len(all_smiles)):
            if i % 10000 == 0:
                print(f"  Processed {i}/{len(all_smiles)} strings...")
            smiles_dset[i] = all_smiles[i] if all_smiles[i] is not None else ""
            selfies_dset[i] = all_selfies[i] if all_selfies[i] is not None else ""
    
    end_time = time.time()
    print(f"Successfully saved to {filename} in {end_time - start_time:.2f} seconds")

def load_molecule_hdf5(filename, idx):
    """Load single molecule from HDF5 file"""
    with h5py.File(filename, 'r') as f:
        # Get coordinates and reshape
        coords_flat = f['coords'][idx]
        coords = coords_flat.reshape(-1, 3)
        
        # Get atom types (decode bytes to string)
        atom_types_str = f['atom_types'][idx]
        if isinstance(atom_types_str, bytes):
            atom_types_str = atom_types_str.decode('utf-8')
        atom_types = atom_types_str.split('|')
        
        # Get strings (decode bytes to string if necessary)
        smiles = f['smiles'][idx]
        if isinstance(smiles, bytes):
            smiles = smiles.decode('utf-8')
            
        selfies = f['selfies'][idx]
        if isinstance(selfies, bytes):
            selfies = selfies.decode('utf-8')
        
        return {
            'atom_types': atom_types,
            'coords': coords,
            'smiles': smiles,
            'selfies': selfies
        }

def get_dataset_info_hdf5(filename):
    """Get information about the HDF5 dataset"""
    with h5py.File(filename, 'r') as f:
        n_molecules = len(f['coords'])
        print(f"Dataset contains {n_molecules} molecules")
        
        # Sample first molecule for info
        sample_coords = f['coords'][69]
        
        sample_atoms_str = f['atom_types'][69]
        if isinstance(sample_atoms_str, bytes):
            sample_atoms_str = sample_atoms_str.decode('utf-8')
        sample_atoms = sample_atoms_str.split('|')
        
        sample_smiles = f['smiles'][69]
        if isinstance(sample_smiles, bytes):
            sample_smiles = sample_smiles.decode('utf-8')
        
        print(f"Sample molecule 69:")
        print(f"  Atoms: {sample_atoms}")
        print(f"  Coordinates shape: {sample_coords.reshape(-1, 3).shape}")
        print(f"  SMILES: {sample_smiles}")
        
        return n_molecules

In [6]:
print("Loading original data...")
data = np.load("mol3d_data/molecules_last_.9_million.npz", mmap_mode='r', allow_pickle=True)

# Convert to HDF5
print("Converting to HDF5...")
save_data_hdf5("mol3d_data/molecules3d_million4.h5", 
               data["atom_types"], data["coords"], 
               data["smiles"], data["selfies"])

# Test the converted data
print("\nTesting converted data...")
get_dataset_info_hdf5("mol3d_data/molecules3d_million4.h5")

print("\nConversion complete! You can now use 'molecules_1st_million.h5' for fast access.")

Loading original data...
Converting to HDF5...
Starting to save 899644 molecules to mol3d_data/molecules3d_million4.h5...
Saving coordinates...
  Processed 0/899644 coordinates...
  Processed 10000/899644 coordinates...
  Processed 20000/899644 coordinates...
  Processed 30000/899644 coordinates...
  Processed 40000/899644 coordinates...
  Processed 50000/899644 coordinates...
  Processed 60000/899644 coordinates...
  Processed 70000/899644 coordinates...
  Processed 80000/899644 coordinates...
  Processed 90000/899644 coordinates...
  Processed 100000/899644 coordinates...
  Processed 110000/899644 coordinates...
  Processed 120000/899644 coordinates...
  Processed 130000/899644 coordinates...
  Processed 140000/899644 coordinates...
  Processed 150000/899644 coordinates...
  Processed 160000/899644 coordinates...
  Processed 170000/899644 coordinates...
  Processed 180000/899644 coordinates...
  Processed 190000/899644 coordinates...
  Processed 200000/899644 coordinates...
  Process

In [5]:
# Test access speed
print("\nTesting access speed...")
start = time.time()
for i in [42, 69, 271, 314, 1024]:
    mol = load_molecule_hdf5("mol3d_data/molecules_1st_million.h5", i)
    print(f"Molecule {i}: {len(mol['atom_types'])} atoms, SMILES: {mol['smiles'][:20]}...")
end = time.time()
print(f"Accessed 5 molecules in {end - start:.4f} seconds")


Testing access speed...
Molecule 42: 12 atoms, SMILES: [H]OC(=O)C([H])(O[H]...
Molecule 69: 20 atoms, SMILES: [H]OC(=O)C1=C([H])C(...
Molecule 271: 26 atoms, SMILES: [H]O[C@]1([H])[C@@](...
Molecule 314: 25 atoms, SMILES: [H]c1nc([H])c([C@]2(...
Molecule 1024: 26 atoms, SMILES: [H]c1c([H])c(Cl)c([H...
Accessed 5 molecules in 0.0488 seconds


In [17]:
n = 69
# Convert to required format
mol = load_molecule_hdf5("mol3d_data/molecules3d_million1.h5", problematic_mol_id) # problematic_mol_id <-> n
atoms = mol["atom_types"]
nat = len(atoms)  # Number of atoms
coords = np.array(mol["coords"], dtype=float)  # Coordinates as numpy array

print(f"Structure has {nat} atoms")
print(f"Atom types: {atoms}")

# Map atom types to integers
unique_atoms = list(set(atoms))
atom_type_map = {atom: i+1 for i, atom in enumerate(sorted(unique_atoms))}
print(f"Atom type mapping: {atom_type_map}")
typ = np.array([atom_type_map[atom] for atom in atoms], dtype=int)

# Set symmetry threshold - this determines how precisely atoms must match
# after applying symmetry operations (typical values: 0.01-0.1)
sym_thr = 0.1

sym = sofi.compute(nat, typ, coords, sym_thr)

# Print results
print(f"Symmetry threshold used: {sym_thr}")
print(f"Point Group: {sym.pg}")
print(f"Number of symmetry operations: {sym.n_sym}")
print(f"Number of principal axes: {sym.n_prin_ax}")
print(f"Principal axes:")
for i, axis in enumerate(sym.prin_ax):
    print(f"  Axis {i+1}: [{axis[0]:.6f}, {axis[1]:.6f}, {axis[2]:.6f}]")

print("\nSymmetry operations:")
for i in range(sym.n_sym):
    op, n, p, axis, angle = sofi.analmat(sym.matrix[i])
    print(f"  {i+1}: {op}{n}^{p}, axis=[{axis[0]:.3f}, {axis[1]:.3f}, {axis[2]:.3f}], angle={angle:.3f}")
    
print(f"Geometric center: {np.mean(coords, axis=0)}")
print(f"\nHausdorff distances: {sym.dHausdorff}")

# IMPORTANT: sofi.compute() does NOT automatically check for linearity!
# We need to do this manually and correct the point group if needed
is_collinear, linear_axis = sofi.check_collinear(nat, coords)
if is_collinear:
    print(f"\n*** LINEAR MOLECULE DETECTED ***")
    print(f"Structure is linear along axis: [{linear_axis[0]:.3f}, {linear_axis[1]:.3f}, {linear_axis[2]:.3f}]")
    
    # Correct the point group for linear molecules
    if sym.n_sym == 1:
        sym.pg = "C∞v"  # or "Cnv" as shown in documentation
        print("Corrected Point Group: C∞v (linear molecule without inversion)")
    else:
        sym.pg = "D∞h"  # or "Dnh" as shown in documentation  
        print("Corrected Point Group: D∞h (linear molecule with inversion)")
else:
    print(f"\nMolecule is not linear - point group {sym.pg} is correct")

Structure has 15 atoms
Atom types: ['C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
Atom type mapping: {'C': 1, 'H': 2}
Symmetry threshold used: 0.1
Point Group: Cs
Number of symmetry operations: 2
Number of principal axes: 1
Principal axes:
  Axis 1: [0.327376, 0.944867, 0.007099]

Symmetry operations:
  1: E0^1, axis=[1.000, 0.000, 0.000], angle=0.000
  2: S0^1, axis=[0.327, 0.945, 0.007], angle=0.000
Geometric center: [0.00562002 0.00013999 0.00528   ]

Hausdorff distances: [8.00593208e-16 2.93250366e-02]

Molecule is not linear - point group Cs is correct


In [85]:
import random
random.seed(42)

dataloc = "mol3d_data/molecules3d_million1.h5"
with h5py.File(dataloc, 'r') as f:
    n_molecules = len(f['atom_types'])
    print(f"File has {n_molecules} entries")
          
n_samples = 100000
sample_ids = random.sample(range(0, n_molecules), n_samples)

File has 999993 entries


In [10]:
mil = 4
dataloc = f"mol3d_data/molecules3d_million_{mil}.h5"
with h5py.File(dataloc, 'r') as f:
    n_molecules = len(f['atom_types'])
    sample_ids = list(range(n_molecules))
    print(f"File has {n_molecules} entries")
n_samples = n_molecules

File has 899644 entries


In [11]:
max_sym_thr = 1.4
min_sym_thr = 0.4
sym_thr = 0.4
point_groups = []
failed_ids = []

for si in sample_ids:
    
    if (si+1) % 10000 == 0:
        print(f"Processing molecule {si+1}/{n_molecules}")
    
    try:
        mol = load_molecule_hdf5(dataloc, si)
        atoms = mol["atom_types"]
        nat = len(atoms)  # Number of atoms
        coords = np.array(mol["coords"], dtype=float)  # Coordinates as numpy array

        #sym_thr = max_sym_thr - (max_sym_thr - min_sym_thr) * np.exp(-(nat - 2) / 30)
        
        # Map atom types to integers
        unique_atoms = list(set(atoms))
        atom_type_map = {atom: i+1 for i, atom in enumerate(sorted(unique_atoms))}
        typ = np.array([atom_type_map[atom] for atom in atoms], dtype=int)
        
        sym = sofi.compute(nat, typ, coords, sym_thr)
        
        is_collinear, linear_axis = sofi.check_collinear(nat, coords)
        if is_collinear:
            # Correct the point group for linear molecules
            if sym.n_sym == 1:
                sym.pg = "C∞v"  # or "Cnv" as shown in documentation
            else:
                sym.pg = "D∞h"  # or "Dnh" as shown in documentation  
        
        point_groups.append(sym.pg)
        
    except Exception as e:
        # Catch the error and log the molecule ID
        failed_ids.append(si)
        point_groups.append('failed')
        print(f"Molecule {si} failed: {e}")
        continue  # skip to next molecule

Processing molecule 10000/899644
Processing molecule 20000/899644
Processing molecule 30000/899644
Processing molecule 40000/899644
Processing molecule 50000/899644
Processing molecule 60000/899644
Processing molecule 70000/899644
Processing molecule 80000/899644
Processing molecule 90000/899644
Processing molecule 100000/899644
Processing molecule 110000/899644
Processing molecule 120000/899644
Processing molecule 130000/899644
Processing molecule 140000/899644
Processing molecule 150000/899644
Processing molecule 160000/899644
Processing molecule 170000/899644
Processing molecule 180000/899644
Processing molecule 190000/899644
Processing molecule 200000/899644
Processing molecule 210000/899644
Processing molecule 220000/899644
Processing molecule 230000/899644
Processing molecule 240000/899644
Processing molecule 250000/899644
Processing molecule 260000/899644
Processing molecule 270000/899644
Processing molecule 280000/899644
Processing molecule 290000/899644
Processing molecule 300

In [43]:
sym_thr = 0.3
perturbation_scale = 1e-1
point_groups_with_some_noise = []
np.random.seed(43)  # for reproducibility

for fi in failed_ids:

    try:
        mol = load_molecule_hdf5(dataloc, fi)
        atoms = mol["atom_types"]
        nat = len(atoms)  # Number of atoms
        coords = np.array(mol["coords"], dtype=float)  # Coordinates as numpy array
        
        # Perturb linear fragments slightly by adding a very small 
        # random perpendicular displacement to the straight C atoms spine

        c_mask = np.array([a == 'C' for a in atoms])
        n_c = np.sum(c_mask)
        
        is_collinear, linear_axis = sofi.check_collinear(n_c, coords[:n_c])
        
        if abs(linear_axis[0]) < 0.9:
            helper = np.array([1.0, 0.0, 0.0])
        else:
            helper = np.array([0.0, 1.0, 0.0])

        # first perpendicular vector
        perp1 = np.cross(linear_axis, helper)
        perp1 /= np.linalg.norm(perp1)
        # second perpendicular vector
        perp2 = np.cross(linear_axis, perp1)
        perp2 /= np.linalg.norm(perp2)

        perturbation = np.zeros_like(coords)
        random_offsets = (np.random.randn(n_c, 1) * perp1 + np.random.randn(n_c, 1) * perp2)[:, :3]
        perturbation[:n_c] = random_offsets * perturbation_scale
        
        coords += perturbation
        
        # Map atom types to integers
        unique_atoms = list(set(atoms))
        atom_type_map = {atom: i+1 for i, atom in enumerate(sorted(unique_atoms))}
        typ = np.array([atom_type_map[atom] for atom in atoms], dtype=int)
        
        sym = sofi.compute(nat, typ, coords, sym_thr)

        is_collinear, linear_axis = sofi.check_collinear(nat, coords)
        if is_collinear:
            # Correct the point group for linear molecules
            if sym.n_sym == 1:
                sym.pg = "C∞v"  # or "Cnv" as shown in documentation
            else:
                sym.pg = "D∞h"  # or "Dnh" as shown in documentation  
        print(sym.pg)
        point_groups_with_some_noise.append(sym.pg)
    except:
        pass
print(point_groups_with_some_noise)

D3h
D2d
C1
C2
 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 ERROR: cannot set beta. Structure not properly shifted?
 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 small norm:  0.29999999999999999     
           6
 i=           1 j=           7
           1  0.53950993681103787       0.12969781315053980        3.7446830511381896E-003
           1 -0.74388121194645507        1.4829980199682222E-002  -6.1859465610855223E-002
           1   1.8117913204516458       0.13157408674044763      -0.24495649341547637     
           3  -1.9533733785748513        9.3032713874434125E-002  -9.8376234475643851E-002
           2  -2.6444733799099951      -0.40926730457022320       0.44282374629366256     
           4   2.9904267131686182        4.0132710605119359E-002  -4.1376235842825390E-002
 origin at: /home/turih/IterativeRotationsAssignments/src/sofi_routines.f90 line:         454
 at: /home/turih/IterativeRotationsAssignments/src/sofi_routines.f90 line:         100
      ERROR FROM IRA/SOFI library :

In [14]:
with open(f"point_groups_million_{mil}_sym_thr_0.4_sofi.json", "w") as f:
    json.dump(point_groups, f)

In [15]:
with open(f"failed_ids_million_{mil}_sym_thr_0.4_sofi.json", "w") as f:
    json.dump(failed_ids, f)

In [8]:
point_groups = []
failed_ids = []

# Convert atom symbols to atomic numbers
# pymsym requires atomic numbers, not symbols
periodic_table = {
    'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8,
    'F': 9, 'Ne': 10, 'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15,
    'S': 16, 'Cl': 17, 'Ar': 18, 'K': 19, 'Ca': 20, 'Sc': 21, 'Ti': 22,
    'V': 23, 'Cr': 24, 'Mn': 25, 'Fe': 26, 'Co': 27, 'Ni': 28, 'Cu': 29,
    'Zn': 30, 'Ga': 31, 'Ge': 32, 'As': 33, 'Se': 34, 'Br': 35, 'Kr': 36,
    'Rb': 37, 'Sr': 38, 'Y': 39, 'Zr': 40, 'Nb': 41, 'Mo': 42, 'Tc': 43,
    'Ru': 44, 'Rh': 45, 'Pd': 46, 'Ag': 47, 'Cd': 48, 'In': 49, 'Sn': 50
}

for si in sample_ids:

    if (si+1) % 10000 == 0:
        print(f"Processing molecule {si+1}/{n_molecules}")
    
    mol = load_molecule_hdf5(dataloc, si)
    atoms = mol["atom_types"]
    coords = np.array(mol["coords"], dtype=float)  # Coordinates as numpy array
    
    atomic_numbers = [periodic_table[a] for a in atoms]

    # Find the point group
    try:
        point_groups.append(pymsym.get_point_group(atomic_numbers, coords))
    except Exception as e:
        failed_ids.append(si)
        point_groups.append("failed")
        print(f"Error determining point group: {e}")
        
point_groups = [pg.replace("Cinfv", "C∞v").replace("Dinfh", "D∞h") for pg in point_groups]

Processing molecule 10000/899644
Processing molecule 20000/899644
Processing molecule 30000/899644
Processing molecule 40000/899644
Processing molecule 50000/899644
Processing molecule 60000/899644
Processing molecule 70000/899644
Processing molecule 80000/899644
Processing molecule 90000/899644
Processing molecule 100000/899644
Processing molecule 110000/899644
Processing molecule 120000/899644
Processing molecule 130000/899644
Processing molecule 140000/899644
Processing molecule 150000/899644
Processing molecule 160000/899644
Processing molecule 170000/899644
Processing molecule 180000/899644
Processing molecule 190000/899644
Processing molecule 200000/899644
Processing molecule 210000/899644
Processing molecule 220000/899644
Processing molecule 230000/899644
Processing molecule 240000/899644
Processing molecule 250000/899644
Processing molecule 260000/899644
Processing molecule 270000/899644
Processing molecule 280000/899644
Processing molecule 290000/899644
Processing molecule 300

In [9]:
with open(f"point_groups_million_{mil}_pymsym.json", "w") as f:
    json.dump(point_groups, f)